In [18]:
import json
import import_ipynb

In [19]:
from nltk_utils import tokenize, stem, bag_of_words

In [20]:
import numpy as np

In [21]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

In [22]:
from model import NeuralNet

In [42]:
import tensorflow as tf
from tensorflow.keras import layers, models, optimizers, losses

In [52]:
import pickle

In [23]:
with open('intents.json', 'r') as f:
    intents = json.load(f)

print(intents)

{'intents': [{'tag': 'greeting', 'patterns': ['Hi', 'Hey', 'How are you', 'Is anyone there?', 'Hello', 'Good day'], 'responses': ['Hey :-)', 'Hello, thanks for visiting', 'Hi there, what can I do for you?', 'Hi there, how can I help?']}, {'tag': 'goodbye', 'patterns': ['Bye', 'See you later', 'Goodbye'], 'responses': ['See you later, thanks for visiting', 'Have a nice day', 'Bye! Come back again soon.']}, {'tag': 'thanks', 'patterns': ['Thanks', 'Thank you', "That's helpful", "Thank's a lot!"], 'responses': ['Happy to help!', 'Any time!', 'My pleasure']}, {'tag': 'items', 'patterns': ['Which items do you have?', 'What kinds of items are there?', 'What do you sell?'], 'responses': ['We sell coffee and tea', 'We have coffee and tea']}, {'tag': 'payments', 'patterns': ['Do you take credit cards?', 'Do you accept Mastercard?', 'Can I pay with Paypal?', 'Are you cash only?'], 'responses': ['We accept VISA, Mastercard and Paypal', 'We accept most major credit cards, and Paypal']}, {'tag': 'd

In [24]:
all_words = []
tags = []
xy = []

In [25]:
for intent in intents['intents']:
    tag = intent['tag']
    tags.append(tag)
    for pattern in intent['patterns']:
        w = tokenize(pattern)
        all_words.extend(w)
        xy.append((w, tag))

In [26]:
ignore_words = ['?', '!', '.', ',']

In [27]:
print(all_words)

['Hi', 'Hey', 'How', 'are', 'you', 'Is', 'anyone', 'there', '?', 'Hello', 'Good', 'day', 'Bye', 'See', 'you', 'later', 'Goodbye', 'Thanks', 'Thank', 'you', 'That', "'s", 'helpful', 'Thank', "'s", 'a', 'lot', '!', 'Which', 'items', 'do', 'you', 'have', '?', 'What', 'kinds', 'of', 'items', 'are', 'there', '?', 'What', 'do', 'you', 'sell', '?', 'Do', 'you', 'take', 'credit', 'cards', '?', 'Do', 'you', 'accept', 'Mastercard', '?', 'Can', 'I', 'pay', 'with', 'Paypal', '?', 'Are', 'you', 'cash', 'only', '?', 'How', 'long', 'does', 'delivery', 'take', '?', 'How', 'long', 'does', 'shipping', 'take', '?', 'When', 'do', 'I', 'get', 'my', 'delivery', '?', 'Tell', 'me', 'a', 'joke', '!', 'Tell', 'me', 'something', 'funny', '!', 'Do', 'you', 'know', 'a', 'joke', '?']


In [28]:
all_words = [stem(w) for w in all_words if w not in ignore_words]
print(all_words)

['hi', 'hey', 'how', 'are', 'you', 'is', 'anyon', 'there', 'hello', 'good', 'day', 'bye', 'see', 'you', 'later', 'goodby', 'thank', 'thank', 'you', 'that', "'s", 'help', 'thank', "'s", 'a', 'lot', 'which', 'item', 'do', 'you', 'have', 'what', 'kind', 'of', 'item', 'are', 'there', 'what', 'do', 'you', 'sell', 'do', 'you', 'take', 'credit', 'card', 'do', 'you', 'accept', 'mastercard', 'can', 'i', 'pay', 'with', 'paypal', 'are', 'you', 'cash', 'onli', 'how', 'long', 'doe', 'deliveri', 'take', 'how', 'long', 'doe', 'ship', 'take', 'when', 'do', 'i', 'get', 'my', 'deliveri', 'tell', 'me', 'a', 'joke', 'tell', 'me', 'someth', 'funni', 'do', 'you', 'know', 'a', 'joke']


In [29]:
all_words = sorted(set(all_words))
tags = sorted(set(tags))
print(tags)

['delivery', 'funny', 'goodbye', 'greeting', 'items', 'payments', 'thanks']


In [30]:
print(all_words)

["'s", 'a', 'accept', 'anyon', 'are', 'bye', 'can', 'card', 'cash', 'credit', 'day', 'deliveri', 'do', 'doe', 'funni', 'get', 'good', 'goodby', 'have', 'hello', 'help', 'hey', 'hi', 'how', 'i', 'is', 'item', 'joke', 'kind', 'know', 'later', 'long', 'lot', 'mastercard', 'me', 'my', 'of', 'onli', 'pay', 'paypal', 'see', 'sell', 'ship', 'someth', 'take', 'tell', 'thank', 'that', 'there', 'what', 'when', 'which', 'with', 'you']


In [31]:
X_train = []
y_train = []

In [32]:
for (pattern_sentence, tag) in xy:
    bag = bag_of_words(pattern_sentence, all_words)
    X_train.append(bag)

    label = tags.index(tag)
    y_train.append(label)   #CrossEntropyLoss

In [33]:
X_train = np.array(X_train)
y_train = np.array(y_train)

In [34]:
X_train

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 1.],
       ...,
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 1.]], shape=(26, 54), dtype=float32)

In [35]:
y_train

array([3, 3, 3, 3, 3, 3, 2, 2, 2, 6, 6, 6, 6, 4, 4, 4, 5, 5, 5, 5, 0, 0,
       0, 1, 1, 1])

In [43]:
#hyperparameters
num_epochs = 1000
batch_size = 8
learning_rate = 0.001
input_size = len(X_train[0])
hidden_size = 8
output_size = len(tags)

In [44]:
def build_model(input_size, hidden_size, output_size):
    model = models.Sequential([
        layers.Input(shape=(input_size,)),
        layers.Dense(hidden_size, activation='relu'),
        layers.Dense(hidden_size, activation='relu'),
        layers.Dense(output_size)  
    ])
    return model

In [45]:
model = build_model(input_size, hidden_size, output_size)

In [46]:
# Loss and Optimizer
loss_fn = losses.SparseCategoricalCrossentropy(from_logits=True)
optimizer = optimizers.Adam(learning_rate=learning_rate)

model.compile(optimizer=optimizer, loss=loss_fn, metrics=['accuracy'])

In [47]:
history = model.fit(
    X_train, y_train,
    batch_size=batch_size,
    epochs=num_epochs,
    verbose=0  # 0 = silent, 1 = progress bar, 2 = one line per epoch
)

In [48]:
for i in range(0, num_epochs, 100):
    print(f"Epoch {i+1}/{num_epochs}, Loss: {history.history['loss'][i]:.4f}")

final_loss = history.history['loss'][-1]
print(f'final loss: {final_loss:.4f}')

Epoch 1/1000, Loss: 1.9366
Epoch 101/1000, Loss: 0.8277
Epoch 201/1000, Loss: 0.0896
Epoch 301/1000, Loss: 0.0216
Epoch 401/1000, Loss: 0.0092
Epoch 501/1000, Loss: 0.0050
Epoch 601/1000, Loss: 0.0029
Epoch 701/1000, Loss: 0.0019
Epoch 801/1000, Loss: 0.0013
Epoch 901/1000, Loss: 0.0009
final loss: 0.0007


In [50]:
# Save the trained model
model.save("chat_model.keras")

In [51]:
data = {
    "input_size": input_size,
    "hidden_size": hidden_size,
    "output_size": output_size,
    "all_words": all_words,
    "tags": tags
}

In [53]:
with open("data.pkl", "wb") as f:
    pickle.dump(data, f)